In [1]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F

# 자신의 Google Drive 마운트하는 코드를 추가하자!
from google.colab import drive

drive.mount('/content/gdrive', force_remount=True)
filepath = '/content/gdrive/MyDrive' + '/Colab Notebooks/csv/'
mnist = np.load(filepath + 'mnist.npz')

x_train = (mnist['x_train'] - np.mean(mnist['x_train'])) / np.std(mnist['x_train'])
y_train = mnist['y_train']
x_test = (mnist['x_test'] - np.mean(mnist['x_train'])) / np.std(mnist['x_train'])
y_test = mnist['y_test']

print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

Mounted at /content/gdrive
(60000, 28, 28) (60000,) (10000, 28, 28) (10000,)


In [2]:
from torch.utils.data import TensorDataset, DataLoader

# Train Data Set
train_dataset = TensorDataset(torch.FloatTensor(x_train), torch.LongTensor(y_train))
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0)

# Test Data Set
test_dataset = TensorDataset(torch.FloatTensor(x_test), torch.LongTensor(y_test))
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=0)

# GPU 사용 여부 확인
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [3]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.cnn = nn.Sequential(
            # Mnist의 입력채널 1
            # 이전의 출력채널과 이번의 입력채널의 값이 동일해야함.
            nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1),
            nn.ReLU(True),
            nn.Conv2d(16, 16, kernel_size=3, stride=1, padding=1),
            nn.ReLU(True),
            nn.MaxPool2d(2, stride=2),
            nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(True),
            nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(True),
            nn.MaxPool2d(2, stride=2),

            # 추가
            nn.Conv2d(32, 16, kernel_size=3, stride=1, padding=1),
            nn.ReLU(True),
            nn.Conv2d(16, 8, kernel_size=3, stride=1, padding=1),
            nn.ReLU(True),
            nn.MaxPool2d(2, stride=2)
        )
        self.fc1 = nn.Linear(8*3*3, 256)
        self.fc2 = nn.Linear(256, 10) 

    def forward(self, x):
        x = self.cnn(x)
        x = torch.flatten(x, start_dim=1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)

        return x

In [4]:
from torch.optim import Adam

model = CNN().to(device)
criterion = nn.CrossEntropyLoss().to(device)

opti = Adam(model.parameters(), lr=1e-3)

In [5]:
def train(model, dataloader, criterion, data_len, opti):
    correct=0

    for data, target in dataloader:
        data = data.view(-1, 1, 28, 28).to(device)
        target = target.to(device)

        output = model(data)
        loss = criterion(output, target)

        opti.zero_grad()
        loss.backward()
        opti.step()

        pred = output.max(1, keepdim=True)[1]
        correct += pred.eq(target.view_as(pred)).sum().item()

    acc = 100. * correct / data_len
    return acc

In [6]:
def evaluate(model, dataloader, criterion, data_len):
    correct=0

    for data, target in dataloader:
        data = data.view(-1, 1, 28, 28).to(device)
        target = target.to(device)

        output = model(data)
        loss = criterion(output, target)

        pred = output.max(1, keepdim=True)[1]
        correct += pred.eq(target.view_as(pred)).sum().item()

    acc = 100. * correct / data_len
    return acc

In [7]:
epoch=10

for i in range(epoch):
    train_acc = train(model, train_dataloader, criterion, len(train_dataloader.dataset), opti)
    val_acc = evaluate(model, test_dataloader, criterion, len(test_dataloader.dataset))
    print(f"[Epoch: {i:2d}], [Train Acc: {train_acc:3.4f}], [Val Acc: {val_acc:3.4f}]")

[Epoch:  0], [Train Acc: 93.4100], [Val Acc: 98.3200]
[Epoch:  1], [Train Acc: 98.1917], [Val Acc: 98.6000]
[Epoch:  2], [Train Acc: 98.5567], [Val Acc: 98.8900]
[Epoch:  3], [Train Acc: 98.8550], [Val Acc: 98.5700]
[Epoch:  4], [Train Acc: 99.0100], [Val Acc: 99.0400]
[Epoch:  5], [Train Acc: 99.1033], [Val Acc: 98.9000]
[Epoch:  6], [Train Acc: 99.2350], [Val Acc: 99.0800]
[Epoch:  7], [Train Acc: 99.3283], [Val Acc: 99.1600]
[Epoch:  8], [Train Acc: 99.3450], [Val Acc: 99.1200]
[Epoch:  9], [Train Acc: 99.3950], [Val Acc: 99.0800]
